<a href="https://colab.research.google.com/github/wseungjin/2019_cau_oss_hackathon/blob/master/hackathon_team11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
 *  단, 사용할 데이터셋에 따라 is_mnist만 수정
*   모든 구현은 [2. 데이터 전처리]와 [3. 모델 생성]에서만 진행
 *  데이터 전처리 후 트레이닝, 데이터 셋은 x_train_after, x_test_after 변수명을 유지해주세요.
 *  데이터셋이 달라져도 같은 모델 구조를 유지하여야함.
*   [4. 모델 저장]과 [5. 모델 로드 및 평가]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *  트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
 *  team_name을 제외한 다른 부분은 수정하지 말 것
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  모델 구조 파일 (model_structure_teamXX.json)
 *  모델 weight 파일 (model_weight_teamXX.h5)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
* 제출 기한: **오후 6시**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2019_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  두 개의 데이터셋에 대해 다른 모델 구조를 사용한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

import numpy as np

is_mnist = False;

# 데이터셋 로드
# x_train, y_train: 트레이닝 데이터 및 레이블
# x_test, y_test: 테스트 데이터 및 레이블
if is_mnist:
  data_type = 'mnist'
  (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data() # fashion MNIST 데이터셋인 경우,
else:
  data_type = 'cifar10'
  (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data() # cifar10 데이터셋인 경우,


# 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 총 클래스 개수
num_classes = y_test.shape[1]

# 인풋 데이터 타입
input_shape = x_test.shape[1:]

Using TensorFlow backend.


170500096/170498071 [==============================] - 6s 0us/step


# **2. 데이터 전처리**


In [0]:
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0

# mnist일 경우 데이터 형은 [case_num, 28,28]의 형태이다.
# CNN에 사용하기 위해 데이터 형을 [case_num, 28, 28, 1]의 형태로 바꾸어 주어야한다.
if is_mnist:
  x_train_after = x_train_after.reshape(60000,28,28,1)
  x_test_after = x_test_after.reshape(10000,28,28,1)
  input_shape = x_train_after.shape[1:]

In [0]:
save_path = '/content/'
team_name = 'team11'


# val_acc가 best값을 가질 경우 체크포인트로 저장한다.
model_path = save_path +  'model_entire_' + data_type + '_' + team_name + '.h5'
md_checkpoint = keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_acc', verbose=1, save_best_only=True)

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, LeakyReLU, Activation
from keras.layers.normalization import BatchNormalization

# 순차 모델 생성 (가장 기본구조)
model = Sequential()


# 1st hidden layer: CNN layer
# 초기 학습을 빠르게 하기 위하여 glorot_uniform으로 값을 초기화 한다.
# CNN 과정 중에 shrink되기 때문에 패딩을 same으로 해준다.
model.add(Conv2D(32, kernel_size=(3, 3),  padding='same', input_shape=input_shape, kernel_initializer='glorot_uniform'))
# gradient를 스무스하게 하기 위하여 BatchNormalization을 사용하였다.
model.add(BatchNormalization())
# gradient vanishment를 피하기 위하여 LeakyReLU를 사용하였다.
model.add(LeakyReLU(alpha=0.01))

# 2st hidden layer: CNN layer
model.add(Conv2D(32, (3,3), padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

# overfitting 문제를 해결하기 위하여 Dropout 기법을 사용하였다.
model.add(Dropout(0.1))

# 3st hidden layer: CNN layer
model.add(Conv2D(64, kernel_size=(3, 3),  padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

# 4st hidden layer: CNN layer
model.add(Conv2D(64, (3,3),  padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))      
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.2))

# 5st hidden layer: CNN layer
model.add(Conv2D(64, kernel_size=(3, 3),  padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

# 6st hidden layer: CNN layer
model.add(Conv2D(64, (3,3),  padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))      

model.add(Dropout(0.25))

# 7st hidden layer: CNN layer
model.add(Conv2D(64, kernel_size=(3, 3),  padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

# 8st hidden layer: CNN layer
model.add(Conv2D(64, (3,3),  padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))    
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

# 9st hidden layer: CNN layer
model.add(Conv2D(128, kernel_size=(3, 3),  padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))

# 10st hidden layer: CNN layer
model.add(Conv2D(128, (3,3),  padding='same', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.01))          
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Flatten layer: fully connected layer에 사용하기 위하여 1D vector로 만든다.
model.add(Flatten())
          
model.add(Dropout(0.5))

# Output layer: fully-connected layer를 이용하여 output값을 classify한다.
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 128, epochs = 200, shuffle=True, validation_data=[x_test_after, y_test],callbacks = [md_checkpoint])

Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 27s 457us/step - loss: 0.7368 - acc: 0.8066 - val_loss: 0.4082 - val_acc: 0.8777

Epoch 00001: val_acc improved from -inf to 0.87770, saving model to /content/best_model_mnist_team11.h5
Epoch 2/200
60000/60000 [==============================] - 21s 344us/step - loss: 0.4318 - acc: 0.8849 - val_loss: 0.3371 - val_acc: 0.8932

Epoch 00002: val_acc improved from 0.87770 to 0.89320, saving model to /content/best_model_mnist_team11.h5
Epoch 3/200
60000/60000 [==============================] - 21s 346us/step - loss: 0.3462 - acc: 0.9007 - val_loss: 0.2663 - val_acc: 0.9129

Epoch 00003: val_acc improved from 0.89320 to 0.91290, saving model to /content/best_model_mnist_team11.h5
Epoch 4/200
60000/60000 [==============================] - 21s 348us/step - loss: 0.3003 - acc: 0.9091 - val_loss: 0.2696 - val_acc: 0.9104

Epoch 00004: val_acc did not improve from 0.91290
Epoch 5/200
60000/

KeyboardInterrupt: ignored

# **4. 최고의 모델을 가져와 웨이트값과 모델의 구조를 저장**

In [0]:
save_path = '/content/'
team_name = 'team11'

# 최고의 모델을 불러와 웨이트값과 모델의 구조를 저장할 준비를 하기 위해 불러옵니다.
newmodel = keras.models.load_model(save_path + 'model_entire_' + data_type + '_' + team_name + '.h5')


# 모델의 weight 값만 저장합니다.
newmodel.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')

# 모델의 구조만을 저장합니다.
model_json = newmodel.to_json()
with open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)



# **5. 모델 로드 및 평가**

In [0]:
save_path = '/content/'
team_name = 'team11'

#평가합니다
model = keras.models.load_model(save_path + 'model_entire_'+ data_type + '_' + team_name + '.h5')
model.evaluate(x_test_after, y_test)

OSError: ignored